In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hespress/stories_faits-divers.csv
/kaggle/input/hespress/comments_marocains-du-monde.csv
/kaggle/input/hespress/stories_art-et-culture.csv
/kaggle/input/hespress/comments_regions.csv
/kaggle/input/hespress/comments_orbites.csv
/kaggle/input/hespress/stories_regions.csv
/kaggle/input/hespress/comments_art-et-culture.csv
/kaggle/input/hespress/stories_sport.csv
/kaggle/input/hespress/stories_medias.csv
/kaggle/input/hespress/comments_economie.csv
/kaggle/input/hespress/comments_medias.csv
/kaggle/input/hespress/stories_economie.csv
/kaggle/input/hespress/comments_sport.csv
/kaggle/input/hespress/comments_politique.csv
/kaggle/input/hespress/stories_tamazight.csv
/kaggle/input/hespress/stories_societe.csv
/kaggle/input/hespress/stories_orbites.csv
/kaggle/input/hespress/comments_societe.csv
/kaggle/input/hespress/stories_politique.csv
/kaggle/input/hespress/stories_marocains-du-monde.csv
/kaggle/input/hespress/comments_faits-divers.csv
/kaggle/input/hespress/comments_tamazig

**Importing relevant libraries**

In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.isri import ISRIStemmer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import itertools
import random
import os
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
import sklearn
from collections import Counter
from wordcloud import WordCloud

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


**Loading the data**

In [3]:
folder_path = '/kaggle/input/hespress'  

csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

data_frames = {}

for csv_file in csv_files:
    csv_file_path = os.path.join(folder_path, csv_file)
    print(f"Contents of {csv_file}:")
    df = pd.read_csv(csv_file_path)
    df = df.iloc[:, 2:] 
    display(df)
    print("\n") 


Contents of stories_faits-divers.csv:


,title,date,author,story,topic
0,"شهادة طبيب جراح ""تورط"" قنصلا بتر أصبع مسنة",الجمعة 02 أكتوبر 2020 - 20:08,هسبريس من الدار البيضاء,قررت الغرفة الجنائية بمحكمة الاستئناف بمدينة ا...,faits-divers
1,"إحباط تهريب أكثر من طن من ""الحشيش"" بأكادير",الجمعة 02 أكتوبر 2020 - 19:37,رشيد بيجيكن من أكادير,تمكنت المصلحة الولائية للشرطة القضائية بمدينة ...,faits-divers
2,كمين أمن الصويرة يسقط شبكة هجرة غير نظامية,الجمعة 02 أكتوبر 2020 - 18:35,ابراهيم مغراوي من مراكش,أوقفت عناصر فرقة الشرطة القضائية بالمنطقة الإق...,faits-divers
3,"""تلميذة زاكورة"" تتهم زوج أختها باستغلالها جنسيا",الجمعة 02 أكتوبر 2020 - 14:20,هسبريس من زاكورة,كشف مصدر أمني مسؤول أن التلميذة المتحدرة من جم...,faits-divers
4,"""أحجار كريمة مزيفة"" تورط أشخاصا بمدينة أكادير",الجمعة 02 أكتوبر 2020 - 10:05,رشيد بيجيكن من أكادير,تمكنت عناصر المصلحة الولائية للشرطة القضائية ب...,faits-divers
...,...,...,...,...,...
995,مصرع عامل داخل مؤسسة تعليمية قرب شفشاون‎,الخميس 04 يونيو 2020 - 14:05,هسبريس من تطوان,لفظ عامل بناء أنفاسه الأخيرة على الفور، أمس ال...,faits-divers
996,الشرطة تباغت مروجا للمخدرات وسط مدينة تطوان‎,الخميس 04 يونيو 2020 - 12:15,هسبريس من تطوان,تمكنت عناصر المصلحة الولائية للشرطة القضائية ب...,faits-divers
997,تفكيك شبكة مختصة في الإجهاض بمدينة مراكش,الخميس 04 يونيو 2020 - 08:35,إبراهيم مغراوي من مراكش,أحالت عناصر الشرطة القضائية بولاية أمن مراكش، ...,faits-divers
998,"""كوفيد-19"" ينهي حياة ممرض متقاعد في كلميم",الخميس 04 يونيو 2020 - 07:10,مصطفى البكار,أعلنت المديرية الجهوية لوزارة الصحة بجهة كلميم...,faits-divers




Contents of comments_marocains-du-monde.csv:


,comment,score,topic
0,سيذكر التاريخ أن سياح أجانب دخلوا المغرب و موا...,428,marocains-du-monde
1,الحكومات لي كتحتارم شعوبها،رجعت مواطنين الحاصل...,303,marocains-du-monde
2,وايلي حتى فات الفوت عاد سولني كيف ابقيت ويلي و...,154,marocains-du-monde
3,الله اعلم لماذا كل هذا التأخر!!! شكرا على ستة ...,107,marocains-du-monde
4,لقد تأخرت السلطات المغربية كثيرا لإعادة هؤلاء ...,169,marocains-du-monde
...,...,...,...
27548,انسان صادق وطني نمودج يقتدى به. ا قترحه سفيرا ...,27,marocains-du-monde
27549,"Jawad is an example of hard work, discipline a...",12,marocains-du-monde
27550,يقول المثل. من اين ذلك.ااشبل من ذاك الاسد انا ...,11,marocains-du-monde
27551,د. جواد الخراز ممن يجمعون بين تواضع الكبار وشم...,13,marocains-du-monde




Contents of stories_art-et-culture.csv:


...

,title,date,author,story,topic
0,عودة مئات المغربيات العالقات في سبتة ومليلية: ...,الخميس 01 أكتوبر 2020 - 14:00,هسبريس من الرباط,بعد أكثر من ستّة أشهر من الإغلاق لحُدودهما الب...,marocains-du-monde
1,خرموش يدحض افتراءات سفير الجزائر ببلغراد حول ا...,الأربعاء 30 شتنبر 2020 - 05:30,هسبريس من الرباط,"نشرت الجريدة الصربية واسعة الانتشار ""بوليتيكا""...",marocains-du-monde
2,"كشف ""الأخبار الزائفة"" يؤهل شابا مغربيا في برنا...",الأربعاء 30 شتنبر 2020 - 01:30,جمال سماحي من تطوان,تأهل الشاب المغربي وائل أبرون، دكتور مهندس في ...,marocains-du-monde
3,"إسبانيا تمنعُ مغربيات عالقات من ولوج فضاء ""شين...",الثلاثاء 29 شتنبر 2020 - 21:00,هسبريس - عبد السلام الشامخ,لن يكون بمقدورِ المغاربة العالقين في مليلية وس...,marocains-du-monde
4,المغرب يشرع غداً في إعادة مئات المواطنين العال...,الثلاثاء 29 شتنبر 2020 - 11:40,هسبريس - عبد الرحيم العسري,يشرع المغرب، غداً الأربعاء، في إعادة مئات المغ...,marocains-du-monde
...,...,...,...,...,...
995,188 أستاذا لتدريس العربية لأبناء مغاربة العالم,السبت 12 ماي 2018 - 23:25,هسبريس من الرباط,أعلنت وزارة التربية الوطنية والتكوين المهني وا...,marocains-du-monde
996,الحروشي .. قاض مغربي ينتصر للعدل في المحكمة ال...,السبت 12 ماي 2018 - 06:00,هسبريس ـ ط.العاطفي | أ.الخياري,لا يلفظ اسم العربي الحروشي في سلطنة عمان إلا و...,marocains-du-monde
997,غزلان الصيابري .. مغربية تعتلي قمّة المال والأ...,الجمعة 11 ماي 2018 - 06:00,هسبريس ـ ط.العاطفي | أ.الخياري,طال المقام بالمغربية غزلان الصيابري في تجربة ا...,marocains-du-monde
998,هشام تسمارت .. صحافي مغربي ينفتح على الرواية ف...,الخميس 10 ماي 2018 - 06:00,هسبريس ـ ط.العاطفي | أ.الخياري,يعيش هشام تسمارت أواخر سنين العقد الثالث من حي...,marocains-du-monde




Contents of comments_faits-divers.csv:


,comment,score,topic
0,كثرة الهم تضحك ... كيف وصل الاصبوع الى فمه ؟ ل...,-3,faits-divers
1,قنصليات موجودة في المغرب لا تحترم مغاربة في بل...,23,faits-divers
2,قنصل يعض و مواطن يغتصب ويقتل بداعي البحث عن ال...,23,faits-divers
3,كيف دخل المغرب ليزاول مهمة قنصل رغم سوابقه الف...,18,faits-divers
4,شحال من مرة ك نشوف شاحنة عامرة هاكدة ك يبغي يط...,9,faits-divers
...,...,...,...
12748,حماية المستهلك أصبحت من المهام اليومية لهده ال...,6,faits-divers
12749,بإمكانك أن تقضي على جائحة كورونا وجميع الفيروس...,6,faits-divers
12750,هذ الناس المسؤولون ديال دمنات هزو االنيفو بزاف...,0,faits-divers
12751,مسكين مول الماحل امكن غير شي مسكين هدا \nواش م...,5,faits-divers




Contents of comments_tamazight.csv:


,comment,score,topic
0,بعيدا عن بعض التعليق العنصرية ،الامازيغية لغة ...,-36,tamazight
1,كان اكبر خطء ارتكبه المغرب نتيجة الخريف العربي...,66,tamazight
2,كا نتوقع من شي وحدين بلا حشما بلا حشوم غادي قو...,-47,tamazight
3,ارجوا من الاخوة المسؤولين احترام اللغة الأمازي...,-32,tamazight
4,تدريس الامازيغية مضيعة للوقت والمال وفتح باب ا...,55,tamazight
...,...,...,...
59443,انا لا افهم لماذا يصر البعض في الولاء للبعثيين...,9,tamazight
59444,جوابا على ولد كريان لا تقدر ان تناقشني لانك اص...,3,tamazight
59445,جوابا على ولد كريان لا تقدر ان تناقشني لانك اص...,4,tamazight
59446,جوابا على ولد كريان لا تقدر ان تناقشني لانك اص...,2,tamazight


**Text Preprocessing**

In [4]:
def preprocessing_ar(text):
    text = stemming_ar(text)
    text = stopWordRemove_ar(text)
    text = normlizeArabic_ar(text)
    return text

def stopWordRemove_ar(text):
    my_st_file = open("/kaggle/input/stopwords/allstop.txt")
    my_list = my_st_file.read()
    needed_word = []
    words = word_tokenize(text)
    for w in words:
        if w not in my_list:
            needed_word.append(w)
    filtered_sentence = ' '.join(needed_word)
    return filtered_sentence

def stemming_ar(text):
    st = ISRIStemmer()
    stemmed_words = []
    words = word_tokenize(text)
    for w in words:
        stemmed_words.append(st.stem(w))
    stemmed_sentence = ' '.join(stemmed_words)
    return stemmed_sentence

punctuations = '''`1234567890÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

def normlizeArabic_ar(text):
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)

    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    noise = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, "", text)

    return text

def prepareDataSet(df):
    if 'comment' in df.columns and 'story' in df.columns:
        print("Before preprocessing - 'comment' column:")
        print(df['comment'])
        print("\nBefore preprocessing - 'story' column:")
        print(df['story'])
        df['comment'] = df['comment'].apply(lambda x: x if isinstance(x, str) else '')
        df['story'] = df['story'].apply(lambda x: x if isinstance(x, str) else '')
        df['comment'] = df['comment'].apply(preprocessing_ar)
        df['story'] = df['story'].apply(preprocessing_ar)
    elif 'comment' in df.columns:
        print("Before preprocessing - 'comment' column:")
        print(df['comment'])
        df['comment'] = df['comment'].apply(lambda x: x if isinstance(x, str) else '')
        df['comment'] = df['comment'].apply(preprocessing_ar)
    elif 'story' in df.columns:
        print("Before preprocessing - 'story' column:")
        print(df['story'])
        df['story'] = df['story'].apply(lambda x: x if isinstance(x, str) else '')
        df['story'] = df['story'].apply(preprocessing_ar)
    else:
        print("No 'comment' or 'story' column found in the DataFrame.")
    return df

def remove_non_strings(df):
    for col in df.columns:
        if col not in ['comment', 'story']:
            continue
        df[col] = df[col].apply(lambda x: x if isinstance(x, str) else '')
    return df

folder_path = '/kaggle/input/hespress'  

csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
preprocessed_dfs = []
for csv_file in csv_files:
    csv_file_path = os.path.join(folder_path, csv_file)
    print(f"Contents of {csv_file}:")
    df = pd.read_csv(csv_file_path)
    df = df.iloc[:, 2:] 
    df = prepareDataSet(df)
    df_cleaned = remove_non_strings(df)
    preprocessed_dfs.append(df_cleaned)
    display(df_cleaned)
    print("\n") 


Contents of stories_faits-divers.csv:
Before preprocessing - 'story' column:
0      قررت الغرفة الجنائية بمحكمة الاستئناف بمدينة ا...
1      تمكنت المصلحة الولائية للشرطة القضائية بمدينة ...
2      أوقفت عناصر فرقة الشرطة القضائية بالمنطقة الإق...
3      كشف مصدر أمني مسؤول أن التلميذة المتحدرة من جم...
4      تمكنت عناصر المصلحة الولائية للشرطة القضائية ب...
                             ...                        
995    لفظ عامل بناء أنفاسه الأخيرة على الفور، أمس ال...
996    تمكنت عناصر المصلحة الولائية للشرطة القضائية ب...
997    أحالت عناصر الشرطة القضائية بولاية أمن مراكش، ...
998    أعلنت المديرية الجهوية لوزارة الصحة بجهة كلميم...
999    أصدر رئيس المجلس الجماعي لدمنات، قرارا إنذاريا...
Name: story, Length: 1000, dtype: object


,title,date,author,story,topic
0,"شهادة طبيب جراح ""تورط"" قنصلا بتر أصبع مسنة",الجمعة 02 أكتوبر 2020 - 20:08,هسبريس من الدار البيضاء,قرر غرف حكم ءنف بمد دار يضء وصل نقش قضه قنصل ف...,faits-divers
1,"إحباط تهريب أكثر من طن من ""الحشيش"" بأكادير",الجمعة 02 أكتوبر 2020 - 19:37,رشيد بيجيكن من أكادير,تمك صلح شرط قضي بمد كادير ضوء دقق وفر صلح دير ...,faits-divers
2,كمين أمن الصويرة يسقط شبكة هجرة غير نظامية,الجمعة 02 أكتوبر 2020 - 18:35,ابراهيم مغراوي من مراكش,عنصر فرق شرط قضي نطق بمد صويره خميس ثلث ربط بش...,faits-divers
3,"""تلميذة زاكورة"" تتهم زوج أختها باستغلالها جنسيا",الجمعة 02 أكتوبر 2020 - 14:20,هسبريس من زاكورة,كشف صدر سءل لمذ تحدر تنزولين اقليم زاكوره تهم ...,faits-divers
4,"""أحجار كريمة مزيفة"" تورط أشخاصا بمدينة أكادير",الجمعة 02 أكتوبر 2020 - 10:05,رشيد بيجيكن من أكادير,تمك عنصر صلح شرط قضي بمد كادير ربعاء ثلث شخص و...,faits-divers
...,...,...,...,...,...
995,مصرع عامل داخل مؤسسة تعليمية قرب شفشاون‎,الخميس 04 يونيو 2020 - 14:05,هسبريس من تطوان,لفظ عمل بنء فاس ور ربعاء هير جدر وسط ءسس بني ر...,faits-divers
996,الشرطة تباغت مروجا للمخدرات وسط مدينة تطوان‎,الخميس 04 يونيو 2020 - 12:15,هسبريس من تطوان,تمك عنصر صلح شرط قضي بمد طن ءزر فرق بحث دخل مي...,faits-divers
997,تفكيك شبكة مختصة في الإجهاض بمدينة مراكش,الخميس 04 يونيو 2020 - 08:35,إبراهيم مغراوي من مراكش,حلت عنصر شرط قضي بول مراكش ربعاء وكل لك سيد ور...,faits-divers
998,"""كوفيد-19"" ينهي حياة ممرض متقاعد في كلميم",الخميس 04 يونيو 2020 - 07:10,مصطفى البكار,دير جهي وزر صحه بجه كلميم دن نشر خصه سجد رصد ر...,faits-divers




Contents of comments_marocains-du-monde.csv:
Before preprocessing - 'comment' column:
0        سيذكر التاريخ أن سياح أجانب دخلوا المغرب و موا...
1        الحكومات لي كتحتارم شعوبها،رجعت مواطنين الحاصل...
2        وايلي حتى فات الفوت عاد سولني كيف ابقيت ويلي و...
3        الله اعلم لماذا كل هذا التأخر!!! شكرا على ستة ...
4        لقد تأخرت السلطات المغربية كثيرا لإعادة هؤلاء ...
                               ...                        
27548    انسان صادق وطني نمودج يقتدى به. ا قترحه سفيرا ...
27549    Jawad is an example of hard work, discipline a...
27550    يقول المثل. من اين ذلك.ااشبل من ذاك الاسد انا ...
27551    د. جواد الخراز ممن يجمعون بين تواضع الكبار وشم...
27552    كان لي الشرف ان اتعرف على د. جواد الخراز في فا...
Name: comment, Length: 27553, dtype: object


,comment,score,topic
0,ذكر ارخ سيح جنب دخل غرب وطن ظلا كلمتر لمد شهر ...,428,marocains-du-monde
1,حكم كتحتارم شعوبهارجعت وطن حصل خرج فشهرمارس بد...,303,marocains-du-monde
2,فات فوت سول قلي عمل حدد تعن ربط تعد فتح حدد ب...,154,marocains-du-monde
3,لمذ اخر شكر شهر,107,marocains-du-monde
4,لقد اخر سلط غرب اعد عيب وعر حكم نظر قاع ريض حد...,169,marocains-du-monde
...,...,...,...
27548,انس وطن ودج قدي قترح سفر تجل ملك ستر ذكي حكس...,27,marocains-du-monde
27549,Jawad is an example of hard work discipline a...,12,marocains-du-monde
27550,يقل اين ذلكااشبل اسد جبل حبب لسد خرز دكتور جو...,11,marocains-du-monde
27551,جود خرز ممن وضع كبر شمل معرفه ذكر كبر علماء ع...,13,marocains-du-monde




Contents of stories_art-et-culture.csv:
Before preprocessing - 'story' column:
0      وجه "بيت الشعر في المغرب" إلى وزير الثقافة وال...
1      في ظلّ استمرار حالة الطوارئ الصحية المرتبطة بج...
2      تشير مشاهدة فيلم قصير ضمن الثلاثية الأخيرة للم...
3      مِن قَلب أيّام "الحَجْر"، رأتِ النّورَ الفصول ...
4      أعلن الفنان المغربيّ سعيد مسكر تخليه عن مبلغ ا...
                             ...                        
995    تستمرّ الآثار الجانبية للانتقال الحكومي الأخير...
996    قال الدكتور المهدي السعيدي، في ندوة حول "جهود ...
997    أجمعت لجنة المسابقة الخاصة بالأفلام المغربية ا...
998    ألوان حيّة ولمسات تعبيرية تحْمل مآسي إنسانية ل...
999    تقوم فرقة مسرح "سفر" بجولة وطنية لتقديم مسرحيت...
Name: story, Length: 1000, dtype: object


...

,title,date,author,story,topic
0,عودة مئات المغربيات العالقات في سبتة ومليلية: ...,الخميس 01 أكتوبر 2020 - 14:00,هسبريس من الرباط,شهر غلق حدد ريه وفق سلط غرب نسق نظر سبن كبر وط...,marocains-du-monde
1,خرموش يدحض افتراءات سفير الجزائر ببلغراد حول ا...,الأربعاء 30 شتنبر 2020 - 05:30,هسبريس من الرباط,نشر جرد صرب وسع نشر بوليتيكا مءخرا قلا سفر ج...,marocains-du-monde
2,"كشف ""الأخبار الزائفة"" يؤهل شابا مغربيا في برنا...",الأربعاء 30 شتنبر 2020 - 01:30,جمال سماحي من تطوان,شاب غرب وءل رون دكتور هندس حاسوب رحل ثلث رحل ...,marocains-du-monde
3,"إسبانيا تمنعُ مغربيات عالقات من ولوج فضاء ""شين...",الثلاثاء 29 شتنبر 2020 - 21:00,هسبريس - عبد السلام الشامخ,قدر غرب ليل وسب حتل دخل نطق شينغ ورب طول مقب...,marocains-du-monde
4,المغرب يشرع غداً في إعادة مئات المواطنين العال...,الثلاثاء 29 شتنبر 2020 - 11:40,هسبريس - عبد الرحيم العسري,غرب ربعاء ءات غرب ليل محتله وذل حلي شهر نظر غل...,marocains-du-monde
...,...,...,...,...,...
995,188 أستاذا لتدريس العربية لأبناء مغاربة العالم,السبت 12 ماي 2018 - 23:25,هسبريس من الرباط,وزر ربي وطن هني بحث علمي قطع ربي وطن درس لغه ...,marocains-du-monde
996,الحروشي .. قاض مغربي ينتصر للعدل في المحكمة ال...,السبت 12 ماي 2018 - 06:00,هسبريس ـ ط.العاطفي | أ.الخياري,لفظ عرب حرش سلط عمن قرن عبر قضي طيل اضه امض ...,marocains-du-monde
997,غزلان الصيابري .. مغربية تعتلي قمّة المال والأ...,الجمعة 11 ماي 2018 - 06:00,هسبريس ـ ط.العاطفي | أ.الخياري,غرب غزل صيبر جرب هجره وصل ثلث ونف قمه عصم بحر ...,marocains-du-monde
998,هشام تسمارت .. صحافي مغربي ينفتح على الرواية ف...,الخميس 10 ماي 2018 - 06:00,هسبريس ـ ط.العاطفي | أ.الخياري,يعش هشم سمر وخر سنن عقد ثلث حيت ختر ربط رحل جر...,marocains-du-monde




Contents of comments_faits-divers.csv:
Before preprocessing - 'comment' column:
0        كثرة الهم تضحك ... كيف وصل الاصبوع الى فمه ؟ ل...
1        قنصليات موجودة في المغرب لا تحترم مغاربة في بل...
2        قنصل يعض و مواطن يغتصب ويقتل بداعي البحث عن ال...
3        كيف دخل المغرب ليزاول مهمة قنصل رغم سوابقه الف...
4        شحال من مرة ك نشوف شاحنة عامرة هاكدة ك يبغي يط...
                               ...                        
12748    حماية المستهلك أصبحت من المهام اليومية لهده ال...
12749    بإمكانك أن تقضي على جائحة كورونا وجميع الفيروس...
12750    هذ الناس المسؤولون ديال دمنات هزو االنيفو بزاف...
12751    مسكين مول الماحل امكن غير شي مسكين هدا \nواش م...
12752    هذا الخبر اغلب ماجاء فيه كذب وتشويه سمعة المحل...
Name: comment, Length: 12753, dtype: object


,comment,score,topic
0,تضح وصل صبع فمه لبد صرخ تصر خنق,-3,faits-divers
1,قنصل غرب حرم غرب بلد قنصل امر دلل لذز غيف,23,faits-divers
2,قنصل يعض وطن غصب قتل بدع بحث كنز خين كلن لجا ب...,23,faits-divers
3,دخل غرب زول همه قنصل رغم سبق ضاءيه ربم اعن سلط...,18,faits-divers
4,شحل نشف شحن هكد يبغ يطح تبن لنه عقب ههه,9,faits-divers
...,...,...,...
12748,حمي هلك هام لهد لجن حله ركز وعي صحه خصص نتج جر...,6,faits-divers
12749,بام تقض جءح كورو يرس وبء زمن قصر حيي دوي مصل ت...,6,faits-divers
12750,ناس سءل ديل دمن هزو يفو بزف تقد نظف محل باستم...,0,faits-divers
12751,مسك مول احل مسك هدا واش جلس رقب محل جره يمل صح...,5,faits-divers




Contents of comments_tamazight.csv:
Before preprocessing - 'comment' column:
0        بعيدا عن بعض التعليق العنصرية ،الامازيغية لغة ...
1        كان اكبر خطء ارتكبه المغرب نتيجة الخريف العربي...
2        كا نتوقع من شي وحدين بلا حشما بلا حشوم غادي قو...
3        ارجوا من الاخوة المسؤولين احترام اللغة الأمازي...
4        تدريس الامازيغية مضيعة للوقت والمال وفتح باب ا...
                               ...                        
59443    انا لا افهم لماذا يصر البعض في الولاء للبعثيين...
59444    جوابا على ولد كريان لا تقدر ان تناقشني لانك اص...
59445    جوابا على ولد كريان لا تقدر ان تناقشني لانك اص...
59446    جوابا على ولد كريان لا تقدر ان تناقشني لانك اص...
59447    لقد اثبتت تجارب الشعوب ان الدستور الديموقراطي ...
Name: comment, Length: 59448, dtype: object


,comment,score,topic
0,عنصر الامازيغيه لغه ثرث عرق ملك غرب يجب رقي ثرث,-36,tamazight
1,كبر خطء ركب غرب نتج خرف عرب عرف لهج غرب كلغ ءس...,66,tamazight
2,وقع وحد حشم حشم غدي قولو عطت امازيغيه واش ناس ...,-47,tamazight
3,ارج اخه سءل حرم لغه امازيغيه لن دعم كبر لهي تمز,-32,tamazight
4,درس امازيغيه ضيع فتح باب فتن غرب اسس صرع طءل غ...,55,tamazight
...,...,...,...
59443,فهم لمذ يصر ولء بعث قوههج تفا خدم يعر شرق بلد ...,9,tamazight
59444,ولد كري قدر نقش لنك اصل فهم اري فيك غبء سدج نق...,3,tamazight
59445,ولد كري قدر نقش لنك اصل فهم اري فيك غبء سدج نق...,4,tamazight
59446,ولد كري قدر نقش لنك اصل فهم اري فيك غبء سدج نق...,2,tamazight


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

# Combine 'score' and 'comment' columns from DataFrames that have these columns in preprocessed_dfs
combined_dfs = [df[['score', 'comment']] for df in preprocessed_dfs if 'score' in df.columns and 'comment' in df.columns]
combined_df = pd.concat(combined_dfs, ignore_index=True)

X = combined_df['comment']  
y = combined_df['score']   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [47]:
# Feature extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=3000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [49]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
X_train_standardized = scaler.fit_transform(X_train_tfidf)
X_test_standardized = scaler.transform(X_test_tfidf)

 #Apply TruncatedSVD for dimensionality reduction
n_components = 100 
svd = TruncatedSVD(n_components=n_components)
X_train_svd = svd.fit_transform(X_train_standardized)

X_test_svd = svd.transform(X_test_standardized)


In [59]:
hidden_layer_size = 50

y_train_binary = np.where(y_train >= 0, 1, 0)
y_test_binary = np.where(y_test >= 0, 1, 0)

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')
                            ])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(X_train_svd,
          y_train_binary,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks=[early_stopping],
          validation_split=0.1,
          verbose=2)

Epoch 1/100
2243/2243 - 7s - loss: 0.4858 - accuracy: 0.7924 - val_loss: 0.4862 - val_accuracy: 0.7902 - 7s/epoch - 3ms/step
Epoch 2/100
2243/2243 - 6s - loss: 0.4731 - accuracy: 0.7966 - val_loss: 0.4804 - val_accuracy: 0.7917 - 6s/epoch - 2ms/step
Epoch 3/100
2243/2243 - 6s - loss: 0.4689 - accuracy: 0.7980 - val_loss: 0.4816 - val_accuracy: 0.7918 - 6s/epoch - 3ms/step
Epoch 4/100
2243/2243 - 6s - loss: 0.4657 - accuracy: 0.7989 - val_loss: 0.4835 - val_accuracy: 0.7915 - 6s/epoch - 2ms/step


In [61]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_svd, y_test_binary)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

1947/1947 [==============================] - 4s 2ms/step - loss: 0.4759 - accuracy: 0.7959
Test Loss: 0.4758664071559906, Test Accuracy: 0.795852541923523


In [67]:
# After training, make predictions on the test set
y_pred_probs = model.predict(X_test_svd)

# Threshold predicted probabilities to get binary class labels (0 or 1)
y_pred_labels = np.where(y_pred_probs >= 0.5, 1, 0)

# Convert y_test_binary to binary form (0 or 1)
y_test_binary_labels = np.where(y_test_binary >= 0.5, 1, 0)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test_binary_labels, y_pred_labels)
recall = recall_score(y_test_binary_labels, y_pred_labels)
f1 = f1_score(y_test_binary_labels, y_pred_labels)

# Print the metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


1947/1947 [==============================] - 3s 2ms/step
Precision: 0.805444369063772
Recall: 0.9743931715124033
F1-score: 0.8819001457793624
